In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
from src.predictive_models import (
    load_datasets,
    build_frequency_model,
    build_sensationalism_model,
    build_malicious_account_model,
    build_naive_realism_model
)   

from src.articles import (
    fetch_article,
    prepare_article_for_models
)

# Load Datasets

In [4]:
sys.path.append(os.path.abspath(".."))
train_path = "../data/train2.tsv"
val_path = "../data/val2.tsv"
test_path = "../data/test2.tsv"
df_train, df_val, df_test = load_datasets(
    train_path, val_path, test_path
) 



# Load Models

### Frequency Heuristic

In [5]:
train_freq = df_train.copy()
val_freq   = df_val.copy()
test_freq  = df_test.copy()

model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq = build_frequency_model(train_freq)

### Sensationalism

In [6]:
train_sens = df_train.copy()
val_sens  = df_val.copy()
test_sens = df_test.copy()

sens_pipeline, sens_preproc, sens_meta, sens_num = build_sensationalism_model(train_sens, val_sens, test_sens)


### Malicious Account

In [ ]:
train_mal = df_train.copy()
val_mal   = df_val.copy()
test_mal  = df_test.copy()

model_malicious, tfidf_malicious, le_malicious = build_malicious_account_model(train_mal)


### Naive Realism

In [10]:
train_nr = df_train.copy()
val_nr   = df_val.copy()
test_nr  = df_test.copy()

naive_pipeline, naive_preproc, naive_meta, naive_num = build_naive_realism_model(train_nr, val_nr, test_nr)
